In [17]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np


FIRST CREATING A CUSTOM CLASS FOR ONLY FRONT IMAGES

In [18]:

class FrontFootDataset(Dataset):
    def __init__(self, root_dir, phase, transform=None):
        self.root_dir = os.path.join(root_dir, phase)
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {'Normal': 0, 'DFC': 1, 'Controlled': 2}
        
        for label in self.class_to_idx:
            class_dir = os.path.join(self.root_dir, label, 'Front')
            for img_name in os.listdir(class_dir):
                if img_name.endswith(('.png', '.jpg', '.jpeg')):  # Adjust based on your image format
                    self.image_paths.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[label])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [19]:
root_dir = '/kaggle/input/diabeticfootimages/Dataset_cleaned'  

In [20]:
# Image transformations
data_transforms = {
    'Train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [21]:
image_datasets = {
    'train': FrontFootDataset(root_dir, 'Train', transform=data_transforms['Train']),
    'val': FrontFootDataset(root_dir, 'Validation', transform=data_transforms['Validation'])
}
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False)
}
class_names = list(image_datasets['train'].class_to_idx.keys())

# RESNET

In [22]:
def get_model(model_name='resnet'):
    if model_name == 'resnet':
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, len(class_names))
    elif model_name == 'vgg':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, len(class_names))
    return model

# Finetuning Resnet

In [23]:
def train_model(model, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    
    return model

In [24]:
# Initialize and train models
resnet_model = get_model('resnet')

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 156MB/s] 


In [28]:
trained_resnet = train_model(resnet_model, criterion, resnet_optimizer)

Epoch 0/9
train Loss: 0.9411 Acc: 0.5055
val Loss: 0.9723 Acc: 0.4737
Epoch 1/9
train Loss: 0.7856 Acc: 0.6593
val Loss: 0.9611 Acc: 0.6842
Epoch 2/9
train Loss: 0.6930 Acc: 0.7418
val Loss: 0.8697 Acc: 0.6316
Epoch 3/9
train Loss: 0.5647 Acc: 0.8462
val Loss: 0.7050 Acc: 0.6316
Epoch 4/9
train Loss: 0.4538 Acc: 0.8846
val Loss: 0.5741 Acc: 0.6842
Epoch 5/9
train Loss: 0.3904 Acc: 0.8956
val Loss: 0.4605 Acc: 0.7368
Epoch 6/9
train Loss: 0.2869 Acc: 0.9286
val Loss: 0.3844 Acc: 0.7895
Epoch 7/9
train Loss: 0.2181 Acc: 0.9451
val Loss: 0.3119 Acc: 0.8421
Epoch 8/9
train Loss: 0.1825 Acc: 0.9780
val Loss: 0.2563 Acc: 0.9474
Epoch 9/9
train Loss: 0.1401 Acc: 0.9725
val Loss: 0.1911 Acc: 1.0000


In [25]:
criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

In [31]:
# Evaluation and metrics
def evaluate_model(model):
    # Define device within the function
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    
    print(f'Accuracy: {accuracy:.4f}')
    print('Classification Report:')
    print(report)
    print('Confusion Matrix:')
    print(conf_matrix)



In [32]:
# Now evaluate the model
evaluate_model(trained_resnet)

Accuracy: 1.0000
Classification Report:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00         3
         DFC       1.00      1.00      1.00         9
  Controlled       1.00      1.00      1.00         7

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19

Confusion Matrix:
[[3 0 0]
 [0 9 0]
 [0 0 7]]


## CREATE A TEST SET USING IMAGE-2-IMAGE STABLE DIFFUSION MODELS

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
import os

# Load the Stable Diffusion Image-to-Image pipeline
pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v-1-4", torch_dtype=torch.float16)
pipe.to("cuda")

# Define paths
train_images_dir = '/path/to/your/train/images'  # Folder containing training images
output_dir = ''
os.makedirs(output_dir, exist_ok=True)

# Prompts corresponding to the classes
class_prompts = {
    "normal": "a normal foot",
    "dfc": "a diabetic foot image",
    "controlled": "a controlled diabetic foot image"
}

#
for class_name, prompt in class_prompts.items():
    class_path = os.path.join(train_images_dir, class_name)
    output_class_path = os.path.join(output_dir, class_name)
    os.makedirs(output_class_path, exist_ok=True)
    
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        
        # Open and preprocess the image
        init_image = Image.open(img_path).convert("RGB").resize((512, 512))  # Resize to model's expected input size

        # Generate a specified number of variations per image
        for i in range(5):
            # Generate a variation based on the prompt and initial image
            generated_image = pipe(prompt=prompt, init_image=init_image, strength=0.75).images[0]
            
            # Save the generated image
            gen_img_name = f"{os.path.splitext(img_name)[0]}_gen_{i}.png"
            generated_image.save(os.path.join(output_class_path, gen_img_name))

print("Image generation complete.")


# DATA AUGMENTATION

In [ ]:

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),  # Randomly rotate images by up to 15 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color changes
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}